In [1]:
import sys
from flask import Flask, render_template,jsonify,request
from flask_cors import CORS
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
import numpy as np
import pandas as pd

In [2]:
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = "sqlite:///db/inpatient.db"
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)

In [3]:
Base = automap_base(metadata=db.metadata)
engine = db.get_engine()
Base.prepare(engine, reflect=True)
Inpatient = Base.classes.inpatient
Drg = Base.classes.drg

In [4]:
ids = ['190 - CHRONIC OBSTRUCTIVE PULMONARY DISEASE W MCC', '193 - SIMPLE PNEUMONIA & PLEURISY W MCC', '194 - SIMPLE PNEUMONIA & PLEURISY W CC', '291 - HEART FAILURE & SHOCK W MCC', '292 - HEART FAILURE & SHOCK W CC', '392 - ESOPHAGITIS, GASTROENT & MISC DIGEST DISORDERS W/O MCC', '470 - MAJOR JOINT REPLACEMENT OR REATTACHMENT OF LOWER EXTREMITY W/O MCC', '690 - KIDNEY & URINARY TRACT INFECTIONS W/O MCC', '871 - SEPTICEMIA OR SEVERE SEPSIS W/O MV >96 HOURS W MCC', '872 - SEPTICEMIA OR SEVERE SEPSIS W/O MV >96 HOURS W/O MCC']

In [29]:
fils = []
for i in ids:
    response = db.session.query(Inpatient).filter_by(drg_definition = i)
    for r in response:
        rec = r.__dict__.copy()
        del rec['_sa_instance_state']
        fils.append(rec)

In [30]:
df = pd.DataFrame(fils)
df

,average_covered_charges,average_medicare_payments,average_total_payments,drg_definition,hrr_description,lat,lon,mhi,provider_city,provider_id,provider_name,provider_state,provider_street_address,provider_zip_code,total_discharges
0,27820.0,4802.0,5828.0,190 - CHRONIC OBSTRUCTIVE PULMONARY DISEASE W MCC,AL - Dothan,31.140065,-85.398289,38050,DOTHAN,010001,SOUTHEAST ALABAMA MEDICAL CENTER,AL,1108 ROSS CLARK CIRCLE,36301,36
1,16234.0,5266.0,6386.0,190 - CHRONIC OBSTRUCTIVE PULMONARY DISEASE W MCC,AL - Birmingham,34.196159,-86.196898,40086,BOAZ,010005,MARSHALL MEDICAL CENTER SOUTH,AL,2505 U S HIGHWAY 431 NORTH,35957,95
2,29846.0,4780.0,5806.0,190 - CHRONIC OBSTRUCTIVE PULMONARY DISEASE W MCC,AL - Birmingham,NaN,NaN,None,FLORENCE,010006,ELIZA COFFEE MEMORIAL HOSPITAL,AL,205 MARENGO STREET,35631,77
3,11787.0,5362.0,6303.0,190 - CHRONIC OBSTRUCTIVE PULMONARY DISEASE W MCC,AL - Dothan,31.281017,-86.256945,34064,OPP,010007,MIZELL MEMORIAL HOSPITAL,AL,702 N MAIN ST,36467,15
4,32868.0,5790.0,6777.0,190 - CHRONIC OBSTRUCTIVE PULMONARY DISEASE W MCC,AL - Birmingham,33.624485,-86.647907,50539,BIRMINGHAM,010011,ST VINCENT'S EAST,AL,50 MEDICAL PARK EAST DRIVE,35235,62
5,41986.0,5277.0,6567.0,190 - CHRONIC OBSTRUCTIVE PULMONARY DISEASE W MCC,AL - Birmingham,34.382994,-85.803307,41094,FORT PAYNE,010012,DEKALB REGIONAL MEDICAL CENTER,AL,200 MED CENTER DRIVE,35968,46
6,35406.0,5357.0,6618.0,190 - CHRONIC OBSTRUCTIVE PULMONARY DISEASE W MCC,AL - Birmingham,33.215623,-86.797656,67794,ALABASTER,010016,SHELBY BAPTIST MEDICAL CENTER,AL,1000 FIRST STREET NORTH,35007,60
7,23006.0,5073.0,6129.0,190 - CHRONIC OBSTRUCTIVE PULMONARY DISEASE W MCC,AL - Birmingham,34.755494,-87.701265,33869,SHEFFIELD,010019,HELEN KELLER MEMORIAL HOSPITAL,AL,1300 SOUTH MONTGOMERY AVENUE,35660,46
8,12812.0,5568.0,6311.0,190 - CHRONIC OBSTRUCTIVE PULMONARY DISEASE W MCC,AL - Dothan,31.467469,-85.617030,40306,OZARK,010021,DALE MEDICAL CENTER,AL,126 HOSPITAL AVE,36360,19
9,28185.0,6343.0,7813.0,190 - CHRONIC OBSTRUCTIVE PULMONARY DISEASE W MCC,AL - Montgomery,32.269243,-86.201494,42346,MONTGOMERY,010023,BAPTIST MEDICAL CENTER SOUTH,AL,2105 EAST SOUTH BOULEVARD,36116,65


In [31]:
df.count()

average_covered_charges      26418
average_medicare_payments    26418
average_total_payments       26418
drg_definition               26418
hrr_description              26418
lat                          25043
lon                          25043
mhi                          25043
provider_city                26418
provider_id                  26418
provider_name                26418
provider_state               26418
provider_street_address      26418
provider_zip_code            26418
total_discharges             26418
dtype: int64

In [43]:
#the "total_discharges" column has both strings and floats in it. pd.to_numeric has not worked
types = []
for i in df['total_discharges']:
    types.append(type(i))

test = pd.Series(types)
test.value_counts()

<class 'float'>    26381
<class 'str'>         37
dtype: int64

In [48]:
#this is the function I made to try to correct it, but it has not worked yet
def sanitize(data):
    if type(data) == str:
        data.replace(",","")
        data = float(data)
        return data
    if type(data) == float:
        return data
            

for i in df['total_discharges']:
    df['d_scrub'][i] = sanitize(df['total_discharges'][i])
    
df.count()

C:\Users\creatural1\AppData\Local\Continuum\anaconda3\envs\PythonData\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


KeyError: '1,134'

In [35]:
df['d_scrub'].value_counts()

1145    2
1180    2
1102    2
1015    2
1060    2
1316    1
1007    1
1177    1
1073    1
1139    1
1009    1
1412    1
2234    1
1489    1
1297    1
1049    1
1483    1
1472    1
1004    1
1222    1
1274    1
1043    1
1066    1
1150    1
1101    1
1492    1
1271    1
4043    1
1242    1
1867    1
1074    1
1134    1
Name: d_scrub, dtype: int64

In [34]:
df['total_discharges'] = df['total_discharges'].str.replace(",", "")
df['total_discharges'] = pd.to_numeric(df['total_discharges'])
df.dtypes

average_covered_charges      float64
average_medicare_payments    float64
average_total_payments       float64
drg_definition                object
hrr_description               object
lat                          float64
lon                          float64
mhi                           object
provider_city                 object
provider_id                   object
provider_name                 object
provider_state                object
provider_street_address       object
provider_zip_code             object
total_discharges             float64
dtype: object

In [20]:
pd.pivot_table(df, index=['drg_definition', 'hrr_description'], values=['average_covered_charges', 'average_covered_charges', 'average_total_payments', 'total_discharges'])

average_covered_charges  \
drg_definition                                     hrr_description                                        
190 - CHRONIC OBSTRUCTIVE PULMONARY DISEASE W MCC  AK - Anchorage                          53707.200000   
                                                   AL - Birmingham                         33750.800000   
                                                   AL - Dothan                             25289.375000   
                                                   AL - Huntsville                         33448.200000   
                                                   AL - Mobile                             31742.571429   
                                                   AL - Montgomery                         23172.500000   
                                                   AL - Tuscaloosa                         24154.500000   
                                                   AR - Fort Smith                         18763.750000   
                                                   AR - Jonesboro                          22803.500000   
                                                   AR - Little Rock                        23066.050000   
                                                   AR - Springdale                         29343.250000   
                                                   AR - Texarkana                          30621.666667   
                                                   AZ - Mesa                               40366.000000   
                                                   AZ - Phoenix                            36379.095238   
                                                   AZ - Sun City                           42269.500000   
                                                   AZ - Tucson                             33094.666667   
                                                   CA - Alameda County                     77190.750000   
                                                   CA - Bakersfield                        41935.625000   
                                                   CA - Chico                              48675.000000   
                                                   CA - Contra Costa County                99712.833333   
                                                   CA - Fresno                             38320.000000   
                                                   CA - Los Angeles                        61716.258621   
                                                   CA - Modesto                            78429.333333   
                                                   CA - Napa                               46052.000000   
                                                   CA - Orange County                      61134.076923   
                                                   CA - Palm Springs/Rancho Mi             67637.666667   
                                                   CA - Redding                            48714.750000   
                                                   CA - Sacramento                         61739.615385   
                                                   CA - Salinas                            80419.500000   
                                                   CA - San Bernardino                     56791.875000   
...                                                                                                 ...   
872 - SEPTICEMIA OR SEVERE SEPSIS W/O MV >96 HO... UT - Ogden                              24174.000000   
                                                   UT - Provo                              23561.666667   
                                                   UT - Salt Lake City                     22071.071429   
                                                   VA - Arlington                          19906.909091   
                                                   VA - Charlottesville                    19467.000000   
              